# Decision Trees

A decision tree is a non-parametric supervised learning algorithm, which is utilized for both classification and regression tasks. It has a hierarchical, tree structure, which consists of a root node, branches, internal nodes and leaf nodes.

<p align="center">
     <img src="https://www.explorium.ai/wp-content/uploads/2019/12/Decision-Trees-2.png"
          alt="Decission Trees"
          width="500" />
</p>

## *Import Libraries*

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydotplus

from sklearn import metrics, preprocessing, tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from six import StringIO


## *Loading Dataset*
### About the dataset
Imagine that you are a medical researcher compiling data for a study. You have collected data about a set of patients, all of whom suffered from the same illness. During their course of treatment, each patient responded to one of 5 medications, Drug A, Drug B, Drug c, Drug x and y.

Part of your job is to build a model to find out which drug might be appropriate for a future patient with the same illness. The feature sets of this dataset are Age, Sex, Blood Pressure, and Cholesterol of patients, and the target is the drug that each patient responded to.

It is a sample of binary classifier, and you can use the training part of the dataset to build a decision tree, and then use it to predict the class of a unknown patient, or to prescribe it to a new patient.

In [2]:
# Downloading the data from IBM object storage.
!wget -O drug200.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/drug200.csv

--2023-05-11 15:42:21--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/drug200.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6027 (5,9K) [text/csv]
Saving to: 'drug200.csv'

     0K .....                                                 100% 1,40G=0s

2023-05-11 15:42:22 (1,40 GB/s) - 'drug200.csv' saved [6027/6027]



In [3]:
# reading data using pandas df
data = pd.read_csv("drug200.csv", delimiter=',')

# First five rows of data
print(data.head())

# Data size (rows x columns)
print("Data size: ", data.size)

   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  drugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  drugY
Data size:  1200


In [4]:
# data description (statistically)
data.describe()

,Age,Na_to_K
count,200.000000,200.000000
mean,44.315000,16.084485
std,16.544315,7.223956
min,15.000000,6.269000
25%,31.000000,10.445500
50%,45.000000,13.936500
75%,58.000000,19.380000
max,74.000000,38.247000


## *Pre-processing Data*
Using `data` as the `drug200.csv` data read by pandas, we'll declare the following variables:
- `X` as the `feature matrix` (data of `data`)
- `y` as the `response vector (target)`

we'll also need to delete the column that doesn't contain numeric values.

In [5]:
X = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[:5] # first 5 values of X

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.114],
       [28, 'F', 'NORMAL', 'HIGH', 7.798],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

As we can see, some features in this dataset are categorical such as `Sex` or `BP`. Unfortunately, `sklearn - decision trees` doesn't handle categorical variables. However, still we can convert these features to numerical values. `pandas.getdummies()` convert categorical variable into dummy/indicator variables.

In [6]:
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1])

le_BP = preprocessing.LabelEncoder()
le_BP.fit(['LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])

le_Chol = preprocessing.LabelEncoder()
le_Chol.fit(['NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3])

X[:5]

array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.114],
       [28, 0, 2, 0, 7.798],
       [61, 0, 1, 0, 18.043]], dtype=object)

Now we can fill the target variable.

In [7]:
y = data["Drug"]
y.head( )

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

## *Setting up the Decission-Tree model*

We will be using `train/test split` on our `decision tree`.
This `train/test split` will return 4 different parameters. We'll name them:
`Xtrain`, `Xtest`, `y_train`, and `y_test`.

The `train/test split` will need the parameters:
`X`, `y`, `test_size`, and `random_state`.

The `X` and `y` are the arrays requires before the split, the `test_size` represents the ratio of the testing dataset, and `random_state` ensures that we obtain the same splits.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Shape of X_train and y_train
print(f"Shape of X_train: {X_train.shape} and y_train: {y_train.shape}")

# Shape of X_test and y_test
print(f"Shape of X_test: {X_test.shape} and y_test: {y_test.shape}")

Shape of X_train: (140, 5) and y_train: (140,)
Shape of X_test: (60, 5) and y_test: (60,)


## *Modeling*

We'll first create an instance of the `DecissionTreeClassifier` called `drugTree`.
Inside of the classifier, specify *criterion = "entropy"* so we can the information gain of each node.

In [9]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth=4)
drugTree

DecisionTreeClassifier(criterion='entropy', max_depth=4)

Next, we will fit the data with the training feature matrix `X_train` and training response vector `y_train`

In [10]:
drugTree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

## *Prediction*

Let's make some `predictions` on the testing dataset and store it into a variable called `predTree`

In [11]:
predTree = drugTree.predict(X_test)

We can print out `predTree` and `y_test` and visually compare the prediction to the actual values.

In [12]:
for pred in zip(predTree[:5], y_test[:5]):
    print(pred)

('drugY', 'drugY')
('drugX', 'drugX')
('drugX', 'drugX')
('drugC', 'drugC')
('drugY', 'drugY')


## *Evaluation*
Next, let's use `metrics` from `sklearn` and check the accuracy of our model.

In [13]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

DecisionTrees's Accuracy:  0.9833333333333333


`Accuracy classifiacation score` computes subset accuracy: the set of labels predicted for a sample must exactly match teh corresponding set of labels in `y_true`

In multilabel classification, the function returns the subset of accuracy. If the entire set of predicted labels for a sample strictly match the true set of labels, then the subset accuracy is `1.0`; otherwise it is `0.0`.

What if we calculate the `accuracy` score *manually*. Here is how to do it.

In [14]:
y_test.iloc[3]

'drugC'

In [15]:
# First we've to know the size of test set
n = len(y_test)

# Then, we count how many predicted true by the models for each comparison respectively to y_test
c = 0
for i in range(n):
    if predTree[i] == y_test.iloc[i]:
        c += 1

# Last, the accuracy score is c/n
acc = c/n
print("DecisionTrees's Accuracy (calculated manually): ", acc)

DecisionTrees's Accuracy (calculated manually):  0.9833333333333333


## *Visualization*

Let's visualize the tree.

In [16]:
%matplotlib inline

dot_data = StringIO()
filename = "drugTree.png"
featureNames = data.columns[:5]
targetNames = data["Drug"].unique().tolist()
out = tree.export_graphviz(drugTree, feature_names=featureNames, out_file=dot_data, class_names=np.unique(y_train),
                           filled=True, special_characters=True, rotate=False)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img, interpolation='nearest')